# setup Tensorflow

In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

TensorFlow Version: 2.19.0
Keras Version: 3.10.0


# Load the dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [3]:
mnist = keras.datasets.fashion_mnist.load_data()

In [5]:
(x_train, y_train), (x_test, y_test) = mnist
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

# Build a machine learning model

Build a [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model:

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

2025-07-09 12:12:19.488484: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [9]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-1.1529574 , -0.22625375,  0.3071904 ,  0.8516545 ,  0.1195193 ,
        -0.78223705, -0.3654325 , -0.55779076,  0.677091  , -0.16188017]],
      dtype=float32)

The [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to probabilities for each class:

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.03010775, 0.07605723, 0.12966189, 0.2234962 , 0.10747509,
        0.04361944, 0.0661753 , 0.05459537, 0.18769735, 0.08111434]],
      dtype=float32)

**Note:** It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using [losses.SparseCategoricalCrossentropy:](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to - `tf.math.log(1/10) ~= 2.3`.

In [13]:
loss_fn(y_train[:1], predictions).numpy()

2.5118957

Before you start training, configure and compile the model using Keras [Model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile). Set the [optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the loss to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [14]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

# Train and evaluate your model

Use the [Model.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method to adjust your model parameters and minimize the loss

In [15]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2025-07-09 12:27:40.201164: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7555 - loss: 0.6891
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8529 - loss: 0.4089
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8655 - loss: 0.3672
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8709 - loss: 0.3514
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8778 - loss: 0.3341


The [Model.evaluate](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [16]:
model.evaluate(x_test, y_test, verbose=2)

2025-07-09 12:30:11.328872: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


313/313 - 1s - 2ms/step - accuracy: 0.8748 - loss: 0.3498


[0.349826842546463, 0.8748000264167786]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [17]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.84585599e-06, 1.06992346e-07, 3.86761485e-08, 1.00508810e-08,
        2.90111814e-07, 1.37764802e-02, 5.88007833e-06, 5.80279827e-02,
        1.52630964e-05, 9.28171098e-01],
       [8.52565427e-06, 2.73453179e-12, 9.95632350e-01, 1.89646148e-08,
        5.93159464e-04, 4.41302446e-11, 3.76594509e-03, 2.59963004e-13,
        5.05565312e-10, 6.23650640e-14],
       [2.78328471e-08, 1.00000000e+00, 5.50311775e-11, 2.28753816e-08,
        7.85368159e-10, 1.77685304e-16, 2.18009725e-11, 4.36663734e-19,
        4.23278903e-12, 2.11346473e-16],
       [1.65590350e-08, 9.99998331e-01, 6.60396016e-10, 1.71570900e-06,
        5.92132432e-09, 5.53827707e-15, 3.44504647e-10, 2.07214716e-17,
        5.89866871e-12, 6.97401392e-14],
       [1.11608863e-01, 1.05506579e-05, 5.13295345e-02, 2.68913526e-02,
        8.51455890e-03, 2.16663580e-06, 8.00963819e-01, 1.16016627e-05,
        6.66602398e-04, 8.05014679e-07]], dtype=float32)>

# Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the guides. If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).